# Limpieza de Datos

## Limpieza de la variable Teléfono

In [29]:
import pandas as pd
df = pd.read_csv("all_data.csv", dtype={"TELEFONO": str})

In [30]:
# Cuántos valores únicos y cuántos vacíos
print(df["TELEFONO"].nunique(), "valores únicos")
print(df["TELEFONO"].isna().sum(), "NaN / vacíos")
print("Total rows:", df.shape[0])

4215 valores únicos
46 NaN / vacíos
Total rows: 6599


In [31]:
# Valores con notación científica (“.0”)  
mask_sci = df["TELEFONO"].str.contains(r"\.0$", na=False)
print("Con .0 al final:", mask_sci.sum())

# Valores con caracteres no numéricos  
mask_nonnumeric = df["TELEFONO"].str.contains(r"\D", na=False)
print("Con otros símbolos:", mask_nonnumeric.sum())

# Longitudes  
longitudes = df["TELEFONO"].dropna().str.replace(r"\D", "", regex=True).str.len()
print(longitudes.describe())


Con .0 al final: 3522
Con otros símbolos: 3556
count    6553.000000
mean        8.577598
std         0.782789
min         2.000000
25%         8.000000
50%         9.000000
75%         9.000000
max        24.000000
Name: TELEFONO, dtype: float64


In [32]:
import re

def limpiar_telefono(x):
    def clean_number(num_str):
        # 1) manejar notación científica
        try:
            if re.search(r"[eE]", num_str):
                num_str = str(int(float(num_str)))
        except:
            return "-1"
        # 2) quitar todo menos dígitos
        num_str = re.sub(r"\D+", "", num_str)
        if not num_str:
            return "-1"
        # 3) truncar si hay >8 dígitos
        if len(num_str) > 8:
            num_str = num_str[:8]
        # 4) invalidar si <7 dígitos
        if len(num_str) < 7:
            return "-1"
        # 5) asegurar 8 dígitos
        return num_str.zfill(8)

    if pd.isna(x):
        return pd.Series({
            "TELEFONO_LIMPIO": "-1",
            "TELEFONOS_ADICIONALES": ""
        })

    s = str(x).strip()
    # dividir en múltiplos separadores
    partes = re.split(r"[;\-/, ]+", s)

    # limpiar cada parte
    limpiados = [clean_number(p) for p in partes]
    # quedarnos sólo con los válidos
    validos = [n for n in limpiados if n != "-1"]

    if not validos:
        return pd.Series({
            "TELEFONO_LIMPIO": "-1",
            "TELEFONOS_ADICIONALES": ""
        })

    primary = validos[0]
    additional = validos[1:]

    return pd.Series({
        "TELEFONO_LIMPIO": primary,
        "TELEFONOS_ADICIONALES": ";".join(additional)
    })

In [33]:
# Aplicar la función y unir las dos columnas nuevas
df[["TELEFONO_LIMPIO", "TELEFONOS_ADICIONALES"]] = (
    df["TELEFONO"]
      .apply(limpiar_telefono)
)

In [34]:
# Ahora imprimimos las métricas sobre la columna limpia
print((df["TELEFONO_LIMPIO"] == "-1").sum(), "teléfonos inválidos")

51 teléfonos inválidos


In [35]:
print(df[['TELEFONO']].sample(5))

        TELEFONO
2278    78395647
3271  47652631.0
6589  52169634.0
4237  49196605.0
4909  51843372.0


In [38]:
print(df["TELEFONOS_ADICIONALES"])

0        
1        
2        
3        
4        
       ..
6594     
6595     
6596     
6597     
6598     
Name: TELEFONOS_ADICIONALES, Length: 6599, dtype: object


In [37]:
long_post = df["TELEFONO_LIMPIO"]\
    .replace("-1", pd.NA)\
    .dropna()\
    .str.len()\
    .value_counts()
print("Distribución de longitudes tras limpieza:\n", long_post)

Distribución de longitudes tras limpieza:
 TELEFONO_LIMPIO
8    6548
Name: count, dtype: int64


### Razones de las decisiones tomadas para la limpieza:

Decidimos primero normalizar todos los valores a cadenas de dígitos puros para garantizar la comparabilidad y evitar formatos mixtos (espacios, guiones, puntos o notación científica). Al eliminar cualquier carácter no numérico y convertir expresiones en “e” o con decimales a enteros, garantizamos que cada teléfono represente únicamente los dígitos esenciales. Truncar los números a ocho dígitos (o completar con ceros a la izquierda cuando falten) responde a la longitud estándar de los celulares en nuestro contexto, mientras que invalidar aquellas cadenas con menos de siete dígitos evita datos claramente erróneos o incompletos. Además, al extraer el primer número como “principal” y agrupar los adicionales en una columna separada, permitimos tanto el análisis prioritario de un contacto principal como la retención de otras vías de comunicación, sin duplicar registros.